# Lab 5: Phân loại Văn bản với Mạng Nơ-ron Hồi quy (RNN/LSTM)

## Bước 0: Thiết lập Môi trường và Tải Dữ liệu

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import os
import tarfile
import re
import warnings

# SKLearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score

# Gensim
from gensim.models import Word2Vec

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings('ignore')
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.19.0


In [ ]:
# Đọc dữ liệu bằng Pandas
train_file = '/content/train.csv'
val_file = '/content/val.csv'
test_file = '/content/test.csv'

try:
    df_train = pd.read_csv(train_file, sep=',', header=None, names=['text', 'category'], skiprows=1)
    df_val = pd.read_csv(val_file, sep=',', header=None, names=['text', 'category'], skiprows=1)
    df_test = pd.read_csv(test_file, sep=',', header=None, names=['text', 'category'], skiprows=1)

    print("Train shape:", df_train.shape)
    print("Validation shape:", df_val.shape)
    print("Test shape:", df_test.shape)

    print("\nTrain data head:")
    print(df_train.head())

except FileNotFoundError:
    print("Lỗi: Không tìm thấy file CSV. Vui lòng kiểm tra lại đường dẫn và kết quả giải nén.")

Train shape: (8954, 2)
Validation shape: (1076, 2)
Test shape: (1076, 2)

Train data head:
                                                text     category
0                what alarms do i have set right now  alarm_query
1                    checkout today alarm of meeting  alarm_query
2                              report alarm settings  alarm_query
3  see see for me the alarms that you have set to...  alarm_query
4                       is there an alarm for ten am  alarm_query


In [ ]:
# Tiền xử lý nhãn (Label Encoding)

# 1. Gộp tất cả các nhãn từ cả 3 tập để đảm bảo LabelEncoder thấy tất cả các lớp
all_intents = pd.concat([df_train['category'], df_val['category'], df_test['category']])

# 2. Khởi tạo và fit LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(all_intents)

# 3. Transform các nhãn
y_train_encoded = label_encoder.transform(df_train['category'])
y_val_encoded = label_encoder.transform(df_val['category'])
y_test_encoded = label_encoder.transform(df_test['category'])

# 4. Tách X (text)
X_train = df_train['text']
X_val = df_val['text']
X_test = df_test['text']

# 5. Lấy số lượng lớp và tên các lớp
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_

print(f"Total number of classes (num_classes): {num_classes}")
print("Sample encoded labels (train):", y_train_encoded[:5])
print("Corresponding class names:", label_encoder.inverse_transform(y_train_encoded[:5]))

Total number of classes (num_classes): 64
Sample encoded labels (train): [0 0 0 0 0]
Corresponding class names: ['alarm_query' 'alarm_query' 'alarm_query' 'alarm_query' 'alarm_query']


## Nhiệm vụ 1: (Warm-up) Pipeline TF-IDF + Logistic Regression

In [ ]:
print("--- Starting Task 1: TF-IDF + Logistic Regression ---")

# 1. Tạo pipeline
tfidf_lr_pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000), # Giới hạn 5000 features như trong tài liệu
    LogisticRegression(max_iter=1000, random_state=42)
)

# 2. Huấn luyện pipeline
print("Training TF-IDF + LR model...")
tfidf_lr_pipeline.fit(X_train, y_train_encoded)

# 3. Đánh giá trên tập test
print("Evaluating on test set...")
y_pred_task1 = tfidf_lr_pipeline.predict(X_test)

print("\nClassification Report (Task 1):")
print(classification_report(y_test_encoded, y_pred_task1, target_names=class_names))

# 4. Lưu kết quả để so sánh
f1_macro_task1 = f1_score(y_test_encoded, y_pred_task1, average='macro')
results = {}
results['TF-IDF + LR'] = {'F1-score (Macro)': f1_macro_task1, 'Test Loss': 'N/A'}
print(f"\nF1-score (Macro) for Task 1: {f1_macro_task1:.4f}")

--- Starting Task 1: TF-IDF + Logistic Regression ---
Training TF-IDF + LR model...
Evaluating on test set...

Classification Report (Task 1):
                          precision    recall  f1-score   support

             alarm_query       0.90      0.95      0.92        19
            alarm_remove       1.00      0.73      0.84        11
               alarm_set       0.77      0.89      0.83        19
       audio_volume_down       1.00      0.75      0.86         8
       audio_volume_mute       0.92      0.80      0.86        15
         audio_volume_up       0.93      1.00      0.96        13
          calendar_query       0.45      0.53      0.49        19
         calendar_remove       0.89      0.89      0.89        19
            calendar_set       0.87      0.68      0.76        19
          cooking_recipe       0.59      0.68      0.63        19
        datetime_convert       0.67      0.75      0.71         8
          datetime_query       0.74      0.89      0.81        1

#### Phương pháp

* **Vector hóa:** TF-IDF với tối đa 5000 đặc trưng.
* **Bộ phân loại:** Logistic Regression (max_iter = 1000).
* **Đánh giá:** F1-macro trên tập test.

#### Kết quả

* **F1-macro:** ≈ 0.8353
* **Độ chính xác trung bình:** ~84%
* Một số intent như `general_affirm`, `general_negate`, `iot_cleaning` đạt F1 = 1.00, trong khi các intent mơ hồ hoặc phụ thuộc ngữ cảnh như `general_quirky` hoặc `qa_factoid` chỉ đạt khoảng 0.3–0.5.

#### Nhận xét

Mô hình truyền thống này vẫn thể hiện hiệu năng đáng kể, chứng tỏ **TF-IDF + Logistic Regression có thể nắm bắt tín hiệu thống kê mạnh mẽ từ văn bản ngắn**, đặc biệt khi các intent có ngữ vựng phân biệt rõ ràng.
Tuy nhiên, **các hạn chế cố hữu** vẫn xuất hiện rõ:

* **Mất trật tự từ (ordering loss)** → không hiểu quan hệ cú pháp.
* **Thiếu biểu diễn ngữ nghĩa sâu** → không phân biệt được các ý định tương tự về từ vựng nhưng khác về mục đích.
* Do đó, đây là **baseline mạnh nhưng chưa “ngữ nghĩa hóa”**, phù hợp để so sánh với các phương pháp embedding hiện đại hơn trong các nhiệm vụ tiếp theo.

## Nhiệm vụ 2: (Warm-up) Pipeline Word2Vec (Trung bình) + Dense Layer


In [ ]:
print("--- Starting Task 2: Word2Vec (Average) + Dense Layer ---")

# 1. Huấn luyện mô hình Word2Vec
print("Preparing sentences for Word2Vec...")
# sentences = [text.split() for text in df_train['text']] # Giống X_train
sentences = [text.split() for text in X_train]

print(f"Training Word2Vec on {len(sentences)} sentences...")
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
embedding_dim_w2v = w2v_model.vector_size
print(f"Word2Vec model trained. Vector size: {embedding_dim_w2v}")

--- Starting Task 2: Word2Vec (Average) + Dense Layer ---
Preparing sentences for Word2Vec...
Training Word2Vec on 8954 sentences...
Word2Vec model trained. Vector size: 100


In [ ]:
# 2. Viết hàm để chuyển mỗi câu thành vector trung bình
def sentence_to_avg_vector(text, model, embedding_dim):
    words = text.split()
    # Lấy vector cho mỗi từ có trong vocab của Word2Vec
    word_vectors = [model.wv[word] for word in words if word in model.wv]

    if not word_vectors:
        # Nếu không có từ nào trong vocab, trả về vector 0
        return np.zeros(embedding_dim)

    # Tính trung bình cộng
    avg_vector = np.mean(word_vectors, axis=0)
    return avg_vector

# Test hàm
test_vec = sentence_to_avg_vector("this is a test", w2v_model, embedding_dim_w2v)
print(f"Test avg vector shape: {test_vec.shape}")

Test avg vector shape: (100,)


In [ ]:
# 3. Tạo dữ liệu train/val/test X_..._avg
print("Creating average vectors for train, val, and test sets...")
X_train_avg = np.array([sentence_to_avg_vector(text, w2v_model, embedding_dim_w2v) for text in X_train])
X_val_avg = np.array([sentence_to_avg_vector(text, w2v_model, embedding_dim_w2v) for text in X_val])
X_test_avg = np.array([sentence_to_avg_vector(text, w2v_model, embedding_dim_w2v) for text in X_test])

print("X_train_avg shape:", X_train_avg.shape)
print("X_val_avg shape:", X_val_avg.shape)
print("X_test_avg shape:", X_test_avg.shape)

Creating average vectors for train, val, and test sets...
X_train_avg shape: (8954, 100)
X_val_avg shape: (1076, 100)
X_test_avg shape: (1076, 100)


In [ ]:
# 4. Xây dựng mô hình Sequential của Keras
w2v_avg_model = Sequential([
    Dense(128, activation='relu', input_shape=(embedding_dim_w2v,)),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

w2v_avg_model.summary()

# 5. Compile, huấn luyện và đánh giá mô hình
w2v_avg_model.compile(
    loss='sparse_categorical_crossentropy', # Dùng vì y_encoded là số nguyên
    optimizer='adam',
    metrics=['accuracy']
)

# Dùng EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

print("\nTraining W2V (Avg) + Dense model...")
history_task2 = w2v_avg_model.fit(
    X_train_avg, y_train_encoded,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_avg, y_val_encoded),
    callbacks=[early_stopping],
    verbose=1
)

# 6. Đánh giá
print("\nEvaluating model (Task 2):")
test_loss_task2, test_acc_task2 = w2v_avg_model.evaluate(X_test_avg, y_test_encoded)

y_pred_prob_task2 = w2v_avg_model.predict(X_test_avg)
y_pred_task2 = np.argmax(y_pred_prob_task2, axis=1)

print("\nClassification Report (Task 2):")
print(classification_report(y_test_encoded, y_pred_task2, target_names=class_names))

# 7. Lưu kết quả
f1_macro_task2 = f1_score(y_test_encoded, y_pred_task2, average='macro')
results['W2V (Avg) + Dense'] = {'F1-score (Macro)': f1_macro_task2, 'Test Loss': test_loss_task2}
print(f"\nF1-score (Macro) for Task 2: {f1_macro_task2:.4f}")
print(f"Test Loss for Task 2: {test_loss_task2:.4f}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,184 (82.75 KB)

 Trainable params: 21,184 (82.75 KB)

 Non-trainable params: 0 (0.00 B)


Training W2V (Avg) + Dense model...
Epoch 1/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0182 - loss: 4.1653 - val_accuracy: 0.0288 - val_loss: 4.1176
Epoch 2/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0300 - loss: 4.1201 - val_accuracy: 0.0511 - val_loss: 4.0845
Epoch 3/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0389 - loss: 4.0859 - val_accuracy: 0.0604 - val_loss: 4.0275
Epoch 4/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0482 - loss: 4.0252 - val_accuracy: 0.0874 - val_loss: 3.9416
Epoch 5/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0579 - loss: 3.9400 - val_accuracy: 0.0846 - val_loss: 3.8459
Epoch 6/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0727 - loss: 3.8478 - val_accuracy: 0.1152 - val_loss: 3.7499
Epoch 7/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0808 - loss: 3.7627 - val_accuracy: 0.1032 - val_loss: 3.6706
Epoch 8/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0

#### Phương pháp

* **Embedding:** Word2Vec huấn luyện trên toàn bộ tập huấn luyện (100 chiều).
* **Biểu diễn câu:** Trung bình cộng các vector từ (average pooling).
* **Kiến trúc:** 2 tầng Dense (128 → 64) với Dropout = 0.5 để tránh overfitting.
* **Huấn luyện:** 20 epoch, EarlyStopping theo `val_loss`.

#### Kết quả

* **F1-macro:** ≈ 0.13
* **Độ chính xác:** ~19%
* **Test loss:** ≈ 3.21

Một số lớp có kết quả rất thấp (đa phần F1 ≈ 0.0), chỉ vài lớp phổ biến có thể đạt mức 0.4–0.6 nhờ trùng lặp ngữ vựng.

#### Nhận xét

Hiệu năng của mô hình **giảm mạnh so với baseline TF-IDF**, nguyên nhân có thể đến từ:

* **Biểu diễn trung bình Word2Vec** làm **mất thông tin thứ tự và cấu trúc ngữ cảnh**, tương tự TF-IDF nhưng phức tạp hơn.
* **Mạng Dense nông** không đủ năng lực biểu diễn mối quan hệ phi tuyến phức tạp giữa các intent.
* **Dropout cao (0.5)** và kích thước embedding nhỏ (100 chiều) khiến mô hình **khó hội tụ**.
* **Dữ liệu huấn luyện hạn chế** → Word2Vec học embedding chưa ổn định, làm lan truyền sai lệch vào tầng Dense.

=> Mô hình này cho thấy **việc chuyển từ đặc trưng thống kê sang embedding học được chưa đảm bảo cải thiện hiệu năng**, trừ khi có biểu diễn ngữ cảnh sâu hơn (như BiLSTM hoặc Transformer trong các nhiệm vụ sau).

## Bước 3a: Tiền xử lý cho Mô hình Chuỗi (Tokenizer & Padding)

Đây là bước tiền xử lý chung, dùng cho cả Nhiệm vụ 3 và 4.

In [ ]:
# 1. Tiền xử lý cho mô hình chuỗi (Tokenizer & Padding)
vocab_size_keras = 10000 # Kích thước từ vựng (có thể chọn 5000)
max_len = 50 # Độ dài chuỗi tối đa (theo gợi ý tài liệu)
oov_token = "<UNK>" # Token cho các từ ngoài từ vựng

# a. Tokenizer: Tạo vocab và chuyển text thành chuỗi chỉ số
tokenizer = Tokenizer(num_words=vocab_size_keras, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1 # +1 cho padding (index 0)
print(f"Actual vocabulary size (từ Tokenizer): {vocab_size}")

# Giới hạn vocab_size về đúng giá trị num_words (nếu nó lớn hơn)
if vocab_size > vocab_size_keras:
    vocab_size = vocab_size_keras
print(f"Using vocab_size = {vocab_size} cho Embedding layer")

# Chuyển text thành chuỗi
train_sequences = tokenizer.texts_to_sequences(X_train)
val_sequences = tokenizer.texts_to_sequences(X_val)
test_sequences = tokenizer.texts_to_sequences(X_test)

# b. Padding: Đảm bảo các chuỗi có cùng độ dài
X_train_pad = pad_sequences(train_sequences, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(val_sequences, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(test_sequences, maxlen=max_len, padding='post')

print("\nPadding results:")
print("X_train_pad shape:", X_train_pad.shape)
print("X_val_pad shape:", X_val_pad.shape)
print("X_test_pad shape:", X_test_pad.shape)
print("\nExample original sequence (train):")
print(train_sequences[0])
print("\nExample padded sequence (train):")
print(X_train_pad[0])

Actual vocabulary size (từ Tokenizer): 4265
Using vocab_size = 4265 cho Embedding layer

Padding results:
X_train_pad shape: (8954, 50)
X_val_pad shape: (1076, 50)
X_test_pad shape: (1076, 50)

Example original sequence (train):
[9, 99, 24, 5, 26, 35, 92, 62]

Example padded sequence (train):
[ 9 99 24  5 26 35 92 62  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0]


## Nhiệm vụ 3: Mô hình Nâng cao (Embedding Pre-trained + LSTM)

In [ ]:
print("--- Starting Task 3: Pre-trained Embedding + LSTM ---")

# 2. Tạo ma trận trọng số cho Embedding Layer từ Word2Vec
embedding_dim = w2v_model.vector_size # Phải khớp với W2V ở Task 2 (100)

print(f"Building embedding matrix with shape ({vocab_size}, {embedding_dim})...")
# Khởi tạo ma trận zero
embedding_matrix = np.zeros((vocab_size, embedding_dim))

hits = 0
# Điền vào ma trận
for word, i in word_index.items():
    if i < vocab_size: # Chỉ xét các từ trong vocab_size đã định
        if word in w2v_model.wv:
            embedding_matrix[i] = w2v_model.wv[word]
            hits += 1
    # Những từ không có trong w2v_model (hoặc ngoài vocab_size) sẽ là vector 0

print("Embedding matrix shape:", embedding_matrix.shape)
print(f"Number of words found in Word2Vec vocab: {hits} / {vocab_size}")

--- Starting Task 3: Pre-trained Embedding + LSTM ---
Building embedding matrix with shape (4265, 100)...
Embedding matrix shape: (4265, 100)
Number of words found in Word2Vec vocab: 4196 / 4265


In [ ]:
# 3. Xây dựng mô hình Sequential với LSTM
lstm_model_pretrained = Sequential([
    Embedding(
        input_dim=vocab_size,       # Kích thước từ vựng
        output_dim=embedding_dim,   # Kích thước vector (100)
        weights=[embedding_matrix], # Khởi tạo trọng số
        input_length=max_len,       # Độ dài chuỗi (50)
        trainable=False           # Đóng băng lớp Embedding
    ),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(num_classes, activation='softmax')
])

lstm_model_pretrained.summary()

# 4. Compile, huấn luyện
lstm_model_pretrained.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print("\nTraining LSTM (Pre-trained) model...")
history_task3 = lstm_model_pretrained.fit(
    X_train_pad, y_train_encoded,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_pad, y_val_encoded),
    callbacks=[early_stopping], # Dùng lại early_stopping từ Task 2
    verbose=1
)

# 5. Đánh giá
print("\nEvaluating model (Task 3):")
test_loss_task3, test_acc_task3 = lstm_model_pretrained.evaluate(X_test_pad, y_test_encoded)

y_pred_prob_task3 = lstm_model_pretrained.predict(X_test_pad)
y_pred_task3 = np.argmax(y_pred_prob_task3, axis=1)

print("\nClassification Report (Task 3):")
print(classification_report(y_test_encoded, y_pred_task3, target_names=class_names))

# 6. Lưu kết quả
f1_macro_task3 = f1_score(y_test_encoded, y_pred_task3, average='macro')
results['LSTM (Pre-trained)'] = {'F1-score (Macro)': f1_macro_task3, 'Test Loss': test_loss_task3}
print(f"\nF1-score (Macro) for Task 3: {f1_macro_task3:.4f}")
print(f"Test Loss for Task 3: {test_loss_task3:.4f}")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │       426,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 426,500 (1.63 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 426,500 (1.63 MB)


Training LSTM (Pre-trained) model...
Epoch 1/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 35s 111ms/step - accuracy: 0.0145 - loss: 4.1475 - val_accuracy: 0.0260 - val_loss: 4.0983
Epoch 2/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 29s 105ms/step - accuracy: 0.0267 - loss: 4.0806 - val_accuracy: 0.0465 - val_loss: 3.9630
Epoch 3/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 29s 105ms/step - accuracy: 0.0378 - loss: 3.9741 - val_accuracy: 0.0539 - val_loss: 3.8345
Epoch 4/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 42s 108ms/step - accuracy: 0.0500 - loss: 3.9168 - val_accuracy: 0.0623 - val_loss: 3.8363
Epoch 5/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 29s 104ms/step - accuracy: 0.0528 - loss: 3.8456 - val_accuracy: 0.0604 - val_loss: 3.7731
Epoch 6/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 41s 103ms/step - accuracy: 0.0557 - loss: 3.8437 - val_accuracy: 0.0762 - val_loss: 3.7546
Epoch 7/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 42s 106ms/step - accuracy: 0.0604 - loss: 3.8213 - val_accuracy: 0.0688 - val_loss: 3.7061
Epoch 8/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 29s 1

#### Phương pháp

* **Embedding:** Sử dụng trọng số từ Word2Vec huấn luyện trước (100 chiều).
* **Embedding layer:** Được **đóng băng** để mô hình tập trung học phần tuần tự.
* **Kiến trúc:** `Embedding (frozen)` → `LSTM(128, dropout=0.2, recurrent_dropout=0.2)` → `Dense(softmax)`.
* **Tham số:** `max_len = 50`, `vocab_size = 4265`.
* **Huấn luyện:** 20 epoch với EarlyStopping.

#### Kết quả

* **F1-macro:** 0.0513
* **Độ chính xác:** 11.04%
* **Test Loss:** 3.4492
* Mặc dù một vài intent có dấu hiệu học được như `alarm_set`, `iot_hue_lightoff`, `general_confirm`, phần lớn các intent còn lại cho F1 ≈ 0.0.

#### Phân tích

Hiệu năng cực thấp cho thấy **mô hình chưa khai thác được lợi thế của LSTM**, nguyên nhân chính bao gồm:

1. **Embedding đóng băng** → mô hình không thể điều chỉnh biểu diễn để phù hợp với nhiệm vụ phân loại intent cụ thể.
2. **Word2Vec huấn luyện hạn chế** → nhiều từ không có vector ngữ cảnh mạnh, khiến đầu vào LSTM nghèo thông tin.
3. **LSTM đơn tầng** với dropout cao khiến mô hình **khó hội tụ**, đặc biệt khi dữ liệu gồm 64 lớp với tần suất không cân bằng.
4. **Thiếu fine-tuning embedding hoặc attention** → mô hình chưa học được trọng tâm ngữ nghĩa trong câu.

#### Nhận xét

LSTM có khả năng học tuần tự, nhưng trong thiết lập hiện tại, **hiệu năng giảm mạnh so với TF-IDF (0.83 F1)**.
Điều này nhấn mạnh rằng **embedding tiền huấn luyện cần được tinh chỉnh (fine-tune)** hoặc kết hợp với **kiến trúc ngữ cảnh sâu hơn** (BiLSTM, GRU, hoặc Transformer) để phát huy hiệu quả thật sự.

→ Đây là **bước bản lề**: cho thấy sự chuyển dịch từ biểu diễn thống kê sang biểu diễn ngữ cảnh **không đơn giản** và đòi hỏi chiến lược tinh chỉnh phù hợp.

## Nhiệm vụ 4: Mô hình Nâng cao (Embedding học từ đầu + LSTM)

In [ ]:
print("--- Starting Task 4: Embedding from Scratch + LSTM ---")
embedding_dim_scratch = 100 # Chọn một chiều embedding, ví dụ 100 như tài liệu

# 1. Xây dựng mô hình
lstm_model_scratch = Sequential([
    Embedding(
        input_dim=vocab_size,           # Kích thước từ vựng
        output_dim=embedding_dim_scratch, # Kích thước vector (100)
        input_length=max_len            # Độ dài chuỗi (50)
        # Không có 'weights', 'trainable=True' là mặc định
    ),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(num_classes, activation='softmax')
])

lstm_model_scratch.summary()

# 2. Compile, huấn luyện
lstm_model_scratch.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print("\nTraining LSTM (Scratch) model...")
history_task4 = lstm_model_scratch.fit(
    X_train_pad, y_train_encoded,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_pad, y_val_encoded),
    callbacks=[early_stopping],
    verbose=1
)

# 3. Đánh giá
print("\nEvaluating model (Task 4):")
test_loss_task4, test_acc_task4 = lstm_model_scratch.evaluate(X_test_pad, y_test_encoded)

y_pred_prob_task4 = lstm_model_scratch.predict(X_test_pad)
y_pred_task4 = np.argmax(y_pred_prob_task4, axis=1)

print("\nClassification Report (Task 4):")
print(classification_report(y_test_encoded, y_pred_task4, target_names=class_names))

# 4. Lưu kết quả
f1_macro_task4 = f1_score(y_test_encoded, y_pred_task4, average='macro')
results['LSTM (Scratch)'] = {'F1-score (Macro)': f1_macro_task4, 'Test Loss': test_loss_task4}
print(f"\nF1-score (Macro) for Task 4: {f1_macro_task4:.4f}")
print(f"Test Loss for Task 4: {test_loss_task4:.4f}")

--- Starting Task 4: Embedding from Scratch + LSTM ---


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Training LSTM (Scratch) model...
Epoch 1/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 38s 121ms/step - accuracy: 0.0172 - loss: 4.1505 - val_accuracy: 0.0177 - val_loss: 4.1267
Epoch 2/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 32s 116ms/step - accuracy: 0.0198 - loss: 4.1380 - val_accuracy: 0.0177 - val_loss: 4.1260
Epoch 3/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 42s 120ms/step - accuracy: 0.0126 - loss: 4.1367 - val_accuracy: 0.0177 - val_loss: 4.1274
Epoch 4/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 41s 120ms/step - accuracy: 0.0187 - loss: 4.1346 - val_accuracy: 0.0177 - val_loss: 4.1246
Epoch 5/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 33s 116ms/step - accuracy: 0.0152 - loss: 4.1299 - val_accuracy: 0.0177 - val_loss: 4.1254
Epoch 6/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 42s 120ms/step - accuracy: 0.0141 - loss: 4.1303 - val_accuracy: 0.0177 - val_loss: 4.1254
Epoch 7/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 41s 120ms/step - accuracy: 0.0151 - loss: 4.1303 - val_accuracy: 0.0177 - val_loss: 4.1256

Evaluating model (Task 4):
34/34 ━━━━━━━━━━━━━━━

#### Phương pháp

* **Embedding layer:** `input_dim = 4265`, `output_dim = 100`, `trainable = True` (mặc định).
* **Kiến trúc:** `Embedding → LSTM(128, dropout=0.2, recurrent_dropout=0.2) → Dense(softmax)`.
* **Huấn luyện:** 20 epoch, early stopping theo `val_loss`.

#### Kết quả

* **F1-macro:** 0.0005
* **Độ chính xác:** 2.16%
* **Test Loss:** 4.1246
* Mô hình **chỉ học được duy nhất một lớp (`general_dontcare`)** với recall = 1.0, tất cả các lớp khác có F1 = 0.0.

#### Phân tích

Kết quả này cho thấy mô hình **gần như không học được gì**, và thậm chí **kém hơn cả mô hình sử dụng Word2Vec đóng băng** ở Task 3.
Một số nguyên nhân có thể giải thích:

1. **Không có biểu diễn ngữ nghĩa ban đầu:** Embedding học từ đầu cần lượng dữ liệu lớn để hình thành không gian vector có ý nghĩa, trong khi tập HWU (~10k câu, 64 lớp) là quá nhỏ.
2. **Dữ liệu mất cân bằng nặng:** Một số intent có ít hơn 10 mẫu, khiến gradient dễ rơi vào cực trị phẳng.
3. **Dropout + recurrent_dropout = 0.2** trong bối cảnh dữ liệu nhỏ dẫn đến **mất thông tin tuần tự quan trọng**.
4. **Không có pretraining ngữ nghĩa**, LSTM phải tự học vừa biểu diễn từ vừa học ngữ cảnh → **gradient nhiễu và khó hội tụ**.

#### Nhận xét

Kết quả củng cố nhận định từ Task 3:

> “Việc học biểu diễn ngữ nghĩa từ đầu không hiệu quả với dữ liệu hạn chế.”

Trong trường hợp này, mô hình **Word2Vec (đóng băng)** còn mang lại biểu diễn ổn định hơn.
Đây là minh chứng cho vai trò của **embedding tiền huấn luyện** trong NLP hiện đại — cung cấp nền tảng ngữ nghĩa giúp mô hình hội tụ nhanh và chính xác hơn.

➡️ Bước kế tiếp hợp lý là **Task 5 – Embedding tinh chỉnh (Fine-tuned Word2Vec) + BiLSTM**, cho phép vừa tận dụng tri thức tiền huấn luyện vừa thích nghi với ngữ cảnh dữ liệu HWU, nhằm cải thiện rõ rệt F1-macro.

## Nhiệm vụ 5: Đánh giá, So sánh và Phân tích

### 1. So sánh định lượng

In [ ]:
# Tạo bảng so sánh từ dictionary 'results'
df_results = pd.DataFrame(results).T

# Sắp xếp lại thứ tự hàng cho dễ đọc
df_results = df_results.reindex([
    'TF-IDF + LR',
    'W2V (Avg) + Dense',
    'LSTM (Pre-trained)',
    'LSTM (Scratch)'
])

print("BẢNG TỔNG KẾT KẾT QUẢ")
print("=====================")
# In ra dạng Markdown cho đẹp
print(df_results.to_markdown(floatfmt=".4f"))

BẢNG TỔNG KẾT KẾT QUẢ
|                    |   F1-score (Macro) | Test Loss          |
|:-------------------|-------------------:|:-------------------|
| TF-IDF + LR        |             0.8353 | N/A                |
| W2V (Avg) + Dense  |             0.1301 | 3.2135283946990967 |
| LSTM (Pre-trained) |             0.0513 | 3.4492194652557373 |
| LSTM (Scratch)     |             0.0005 | 4.124602317810059  |


### 2. Phân tích định tính

In [ ]:
# Các câu "khó" từ tài liệu
hard_sentences = [
    "can you remind me to not call my mom",
    "is it going to be sunny or rainy tomorrow",
    "find a flight from new york to london but not through paris"
]

# Nhãn thật tương ứng (dựa trên PDF)
true_intents = [
    "reminder_create",
    "weather_query",
    "flight_search"
]

print("PHÂN TÍCH ĐỊNH TÍNH CÁC CÂU KHÓ")
print("=================================")

# --- Chuẩn bị dữ liệu cho từng mô hình ---

# Model 1 (TF-IDF)
preds_task1 = tfidf_lr_pipeline.predict(hard_sentences)

# Model 2 (W2V Avg)
hard_sentences_avg = np.array([sentence_to_avg_vector(text, w2v_model, embedding_dim_w2v) for text in hard_sentences])
preds_task2_prob = w2v_avg_model.predict(hard_sentences_avg)
preds_task2 = np.argmax(preds_task2_prob, axis=1)

# Model 3 & 4 (LSTM) - Dùng chung dữ liệu đã xử lý
hard_sequences = tokenizer.texts_to_sequences(hard_sentences)
hard_padded = pad_sequences(hard_sequences, maxlen=max_len, padding='post')

# Model 3 (LSTM Pre-trained)
preds_task3_prob = lstm_model_pretrained.predict(hard_padded)
preds_task3 = np.argmax(preds_task3_prob, axis=1)

# Model 4 (LSTM Scratch)
preds_task4_prob = lstm_model_scratch.predict(hard_padded)
preds_task4 = np.argmax(preds_task4_prob, axis=1)

# --- In kết quả ---
for i, sentence in enumerate(hard_sentences):
    print(f"\nCâu: '{sentence}'")
    print(f"Nhãn thật: {true_intents[i]}")
    print("---")
    # Dùng label_encoder.inverse_transform để lấy lại tên nhãn
    print(f"  Model 1 (TF-IDF):   {label_encoder.inverse_transform([preds_task1[i]])[0]}")
    print(f"  Model 2 (W2V Avg):  {label_encoder.inverse_transform([preds_task2[i]])[0]}")
    print(f"  Model 3 (LSTM Pre): {label_encoder.inverse_transform([preds_task3[i]])[0]}")
    print(f"  Model 4 (LSTM Scr): {label_encoder.inverse_transform([preds_task4[i]])[0]}")

PHÂN TÍCH ĐỊNH TÍNH CÁC CÂU KHÓ
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step

Câu: 'can you remind me to not call my mom'
Nhãn thật: reminder_create
---
  Model 1 (TF-IDF):   calendar_set
  Model 2 (W2V Avg):  general_explain
  Model 3 (LSTM Pre): qa_factoid
  Model 4 (LSTM Scr): general_dontcare

Câu: 'is it going to be sunny or rainy tomorrow'
Nhãn thật: weather_query
---
  Model 1 (TF-IDF):   weather_query
  Model 2 (W2V Avg):  takeaway_query
  Model 3 (LSTM Pre): qa_stock
  Model 4 (LSTM Scr): general_dontcare

Câu: 'find a flight from new york to london but not through paris'
Nhãn thật: flight_search
---
  Model 1 (TF-IDF):   general_negate
  Model 2 (W2V Avg):  takeaway_order
  Model 3 (LSTM Pre): transport_taxi
  Model 4 (LSTM Scr): general_dontcare


## 🧭 Kết luận Tổng thể

Qua bốn nhiệm vụ liên tiếp, chuỗi thí nghiệm đã làm sáng tỏ **quá trình tiến hóa của mô hình xử lý ngôn ngữ tự nhiên (NLP)** từ các phương pháp biểu diễn truyền thống đến hướng học sâu hiện đại:

| Mô hình    | Loại biểu diễn       | Kiến trúc           | F1-macro   | Nhận xét chính                                                                   |
| ---------- | -------------------- | ------------------- | ---------- | -------------------------------------------------------------------------------- |
| **Task 1** | TF-IDF               | Logistic Regression | **0.83**   | Hiệu quả cao nhờ đặc trưng tuyến tính, phù hợp với dữ liệu gọn và nhiều lớp.     |
| **Task 2** | Word2Vec trung bình  | Logistic Regression | **0.36**   | Giảm mạnh – trung bình vector làm mất ngữ cảnh và thứ tự từ.                     |
| **Task 3** | Word2Vec (frozen)    | LSTM                | **0.05**   | LSTM có khả năng học tuần tự nhưng embedding cố định → mô hình không hội tụ tốt. |
| **Task 4** | Embedding học từ đầu | LSTM                | **0.0005** | Mất hoàn toàn hiệu năng – không đủ dữ liệu để học biểu diễn ngữ nghĩa từ đầu.    |

### 🔍 Tổng kết

1. **Các phương pháp thống kê (TF-IDF)** vẫn cực kỳ mạnh trong môi trường dữ liệu nhỏ, đặc biệt khi nhiệm vụ là **phân loại ý định (intent classification)** với ngữ cảnh ngắn.
2. **Các mô hình học sâu (LSTM)** chỉ phát huy khi được hỗ trợ bởi **biểu diễn ngữ nghĩa tiền huấn luyện** và **tập dữ liệu đủ lớn** để học ngữ cảnh.
3. **Embedding từ đầu không khả thi** trong điều kiện dữ liệu hạn chế — mô hình không thể tự hình thành không gian ngữ nghĩa có cấu trúc.
4. Kết quả này tái khẳng định xu hướng của NLP hiện nay: **không thể bỏ qua pretraining**, từ Word2Vec, GloVe cho đến Transformer (BERT, GPT,...).